In [1]:
import pandas as pd
import numpy as np
import os.path as osp
%load_ext dotenv
%dotenv env.sh
%run -m cytokit_nb.keyence

In [9]:
df = pd.read_csv('experiments-base.csv')
df

,name,dir
0,sta-00.125-20um-s,0125uMsta-20um-grids
1,sta-00.250-20um-s,025uMsta-20um-grids
2,sta-00.500-20um-s,05uMsta-20um-grids
3,sta-01.000-20um-s,1uMsta-20um-grids
4,sta-02.000-20um-s,2uMsta-20um-grids
5,sta-04.000-20um-s,4uMsta-grids
6,sta-00.000-20um-s,untreated-20um-grids
7,sta-00.000-20um-u,untreated-unstained20um-grids


In [19]:
grids = ['XY01', 'XY02', 'XY03']

def add_info(r, grid):
    path = osp.join(os.environ['EXP_GROUP_RAW_DIR'], r['dir'], grid)
    info = analyze_keyence_dataset(path)
    assert info['z_pitch'].nunique() == 1
    base_conf = 'experiment.yaml'
    if r['dir'] == '4uMsta-grids':
        base_conf = 'experiment_cy5last.yaml'
    return r.append(pd.Series({
        'grid': grid,
        'z_pitch': info['z_pitch'].iloc[0],
        'n_tiles': info['tile'].nunique(),
        'n_z': info['z'].nunique(),
        'n_ch': info['ch'].nunique(),
        'chs': tuple(sorted(info['ch'].unique())),
        'conf': base_conf
    })).sort_index()

dfi = pd.DataFrame([
    add_info(r, grid=g)
    for _, r in df.iterrows()
    for g in grids
])
dfi.head()

,chs,conf,dir,grid,n_ch,n_tiles,n_z,name,z_pitch
0,"(1, 2, 3)",experiment.yaml,0125uMsta-20um-grids,XY01,3,27,8,sta-00.125-20um-s,200
1,"(1, 2, 3)",experiment.yaml,0125uMsta-20um-grids,XY02,3,27,7,sta-00.125-20um-s,200
2,"(1, 2, 3)",experiment.yaml,0125uMsta-20um-grids,XY03,3,27,9,sta-00.125-20um-s,200
3,"(1, 2, 3)",experiment.yaml,025uMsta-20um-grids,XY01,3,27,8,sta-00.250-20um-s,200
4,"(1, 2, 3)",experiment.yaml,025uMsta-20um-grids,XY02,3,27,7,sta-00.250-20um-s,200


In [20]:
dfi['chs'].unique()

array([('1', '2', '3'), ('1', '3', '4')], dtype=object)

In [21]:
dfi[dfi['chs'] == ('1', '3', '4')]

,chs,conf,dir,grid,n_ch,n_tiles,n_z,name,z_pitch
15,"(1, 3, 4)",experiment_cy5last.yaml,4uMsta-grids,XY01,3,27,10,sta-04.000-20um-s,200
16,"(1, 3, 4)",experiment_cy5last.yaml,4uMsta-grids,XY02,3,27,9,sta-04.000-20um-s,200
17,"(1, 3, 4)",experiment_cy5last.yaml,4uMsta-grids,XY03,3,27,9,sta-04.000-20um-s,200


In [22]:
dfi[['name', 'grid', 'n_z']]

,name,grid,n_z
0,sta-00.125-20um-s,XY01,8
1,sta-00.125-20um-s,XY02,7
2,sta-00.125-20um-s,XY03,9
3,sta-00.250-20um-s,XY01,8
4,sta-00.250-20um-s,XY02,7
5,sta-00.250-20um-s,XY03,9
6,sta-00.500-20um-s,XY01,7
7,sta-00.500-20um-s,XY02,7
8,sta-00.500-20um-s,XY03,8
9,sta-01.000-20um-s,XY01,7


In [23]:
assert dfi['n_ch'].nunique() == 1
assert dfi['z_pitch'].nunique() == 1
assert dfi['n_tiles'].nunique() == 1

In [24]:
dfe = dfi.copy()
dfe = dfe.rename(columns={'name': 'cond'})
dfe = dfe.drop('chs', axis=1)
dfe.insert(0, 'name', dfe.apply(lambda r: '{}-{}'.format(r['cond'], r['grid']), axis=1))
dfe.head()

,name,conf,dir,grid,n_ch,n_tiles,n_z,cond,z_pitch
0,sta-00.125-20um-s-XY01,experiment.yaml,0125uMsta-20um-grids,XY01,3,27,8,sta-00.125-20um-s,200
1,sta-00.125-20um-s-XY02,experiment.yaml,0125uMsta-20um-grids,XY02,3,27,7,sta-00.125-20um-s,200
2,sta-00.125-20um-s-XY03,experiment.yaml,0125uMsta-20um-grids,XY03,3,27,9,sta-00.125-20um-s,200
3,sta-00.250-20um-s-XY01,experiment.yaml,025uMsta-20um-grids,XY01,3,27,8,sta-00.250-20um-s,200
4,sta-00.250-20um-s-XY02,experiment.yaml,025uMsta-20um-grids,XY02,3,27,7,sta-00.250-20um-s,200


In [25]:
dfe.to_csv('experiments.csv', index=False)